# LABORATORIO #2 - Deteccion de SPAM
 Alfredo Quezada 191002
 
 Randy Venegas   

* #### Exploracion de datos

In [1]:
# agregamos el data set para poder tener una idea de lo que son los datos. 
import pandas as pd 
df1 = pd.read_csv('completeSpamAssassin.csv')
df1.sample(n=5).head()

,Unnamed: 0,Body,Label
3031,3031,empty,0
3012,3012,"Once upon a time, Jesse wrote :> Have you thou...",0
5080,5080,"""Adam L. Beberg"" writes:> > Interestingly, it...",0
4339,4339,"""Leonard R. Cleavelin"" wrote:\n>Scat I have, u...",0
5561,5561,Another thing I see in debian but not in my RH...,0


In [2]:
# cargamos tambien el segundo conjunto de datos.
df2 = pd.read_csv('enronSpamSubset.csv')
df2.sample(n=5).head()

,Unnamed: 0.1,Unnamed: 0,Body,Label
9036,20810,20810,"Subject: si server will be unavailable , today...",0
7068,31180,31180,"Subject: fw : a weygandt , andrew ; glover , r...",0
5588,28134,28134,"Subject: re :\n shirley ,\n i will take half d...",0
870,5801,5801,"Subject: xanax for sale , no prior prescriptio...",1
4276,9236,9236,Subject: tatts award claims\n tatts lotto\n in...,1


Ahora que ya sabemos lo que son los dos conjuntos de datasets, procedemos a unificar los dos para crear un data set que contenga ambas caracteristicas. 

In [3]:
#Ahora hacemos el data set final 
#Como se puede ver eliminados los datos que no son relevantes
df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0.1'], axis=1, inplace=True)

df = pd.concat([df1, df2], ignore_index=True)
df.head()
#Por ultimo guardamos el nuevo data set en un csv final.
df.to_csv('dbfinal.csv', index=False)
db = pd.read_csv('dbfinal.csv')
db.head()


,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


Con esta parte realizada, ahora podemos pasar a la parte de preprocesamiento

### Preprocesamiento

In [4]:
# Lo primero es elimanar todos los elementos no deseados y poner los datos en el mismo formato 
db['Body'] = db['Body'].astype(str).str.lower()
db.head()

,Body,Label
0,\nsave up to 70% on life insurance.\nwhy spend...,1
1,1) fight the risk of cancer!\nhttp://www.adcli...,1
2,1) fight the risk of cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,i thought you might like these:\n1) slim down ...,1


In [5]:
#Ahora eliminamos los formatos no deseados 
import re
db['Body']= db['Body'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))
db['Body']= db['Body'].apply(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", x))
db.head()

,Body,Label
0,save up to on life insurance why spend more...,1
1,fight the risk of cancer slim down guar...,1
2,fight the risk of cancer slim down guar...,1
3,...,1
4,i thought you might like these slim down g...,1


In [6]:
import nltk
from nltk.tokenize import RegexpTokenizer
#Se procede a tokenizar las palabras
regexp = RegexpTokenizer('\w+')
db['text_token']=db['Body'].apply(regexp.tokenize)
db.head(3)

,Body,Label,text_token
0,save up to on life insurance why spend more...,1,"[save, up, to, on, life, insurance, why, spend..."
1,fight the risk of cancer slim down guar...,1,"[fight, the, risk, of, cancer, slim, down, gua..."
2,fight the risk of cancer slim down guar...,1,"[fight, the, risk, of, cancer, slim, down, gua..."


In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords

# Hace una lista de las stopwords
stopwords = nltk.corpus.stopwords.words("english")

# Extiende las listas con stopwords
my_stopwords = ['https']
stopwords.extend(my_stopwords)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Randy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Body,Label,text_token,text_clean,text_string
0,save up to on life insurance why spend more...,1,"[save, life, insurance, spend, life, quote, sa...",save life insurance spend life quote savings e...,save life insurance spend life quote savings e...
1,fight the risk of cancer slim down guar...,1,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...
2,fight the risk of cancer slim down guar...,1,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...


In [14]:

# Quita las stopwords
db['text_token'] = db['text_token'].apply(lambda x: [item for item in x if item not in stopwords])
db.head(3)
# Quita las palabras infrecuentes 
db['text_clean'] = db['text_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))
db[['Body', 'text_token', 'text_clean']].head()



,Body,text_token,text_string
0,save up to on life insurance why spend more...,"[save, life, insurance, spend, life, quote, sa...",save life insurance spend life quote savings e...
1,fight the risk of cancer slim down guar...,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...
2,fight the risk of cancer slim down guar...,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...
3,...,"[adult, club, offers, free, membership, instan...",adult club offers free membership instant acce...
4,i thought you might like these slim down g...,"[thought, might, like, slim, guaranteed, lose,...",thought might like slim guaranteed lose lbs da...


In [32]:
# Crea una lista con todas las palabras
from nltk.tokenize import *
all_words = ' '.join([word for word in db['text_clean']])





In [34]:
# Tokenize 
tokenized_words = nltk.tokenize.word_tokenize(all_words)




LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Randy/nltk_data'
    - 'c:\\users\\randy\\appdata\\local\\programs\\python\\python38\\nltk_data'
    - 'c:\\users\\randy\\appdata\\local\\programs\\python\\python38\\share\\nltk_data'
    - 'c:\\users\\randy\\appdata\\local\\programs\\python\\python38\\lib\\nltk_data'
    - 'C:\\Users\\Randy\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [30]:
from nltk.probability import FreqDist
fdist = FreqDist(tokenized_words)
# Las tres palabras mas comunes
fdist.most_common(3)

# Las tres palabras más incomunes
fdist.most_common()[-3:]

NameError: name 'tokenized_words' is not defined

In [ ]:
# Quita las palabras comunes
db['text_clean']=  db['text_clean'].str.replace("subject", " ")
db['text_clean']=  db['text_clean'].str.replace("enron", " ")
db['text_clean']=  db['text_clean'].str.replace("com", " ")
db['text_clean']=  db['text_clean'].str.replace("email", " ")

# Vuelve a crear una lista con todas las palabras
all_words = ' '.join([word for word in db['text_clean']])
# Tokenize 
tokenized_words = nltk.tokenize.word_tokenize(all_words)
from nltk.probability import FreqDist

fdist = FreqDist(tokenized_words)
# Vuelve a imprimir las tres mas comunes
fdist.most_common(3)

# Quita las palabras mas frecuentes 
db['text_fdist'] = db['text_token'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 1 ]))
db[['Body', 'text_token', 'text_clean', 'text_fdist']].head()
